# Query Optimization Demo

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Load in the IMDB Performance database

This is a variation of the IMDB database with keys defined. Note that this is a pretty big database! So if you run the below lines, please also remember to delete the `imdb_perf_lecture` afterwards to save space on your limited postgreSQL server.

We assume you have the associated lecture folder `indexes` pulled into your repo already. The below commands create a symbolic link (i.e., shortcut/redirect with `ln`) to this lecture data directory, allowing some space saving, and unzip the database file.

In [3]:
!ln -sf ../../lec/indexes/data .
!unzip -u data/imdb_perf_lecture.zip -d data/

Archive:  data/imdb_perf_lecture.zip


In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb_perf_lecture'
!psql -h localhost -c 'CREATE DATABASE imdb_perf_lecture' 
!psql -h localhost -d imdb_perf_lecture -f data/imdb_perf_lecture.sql

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 845888
COPY 2211936
COPY 656453
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE


## Start `jupysql`

In [5]:
%reload_ext sql

In [6]:
%sql postgresql://127.0.0.1:5432/imdb_perf_lecture

Connecting to 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

If you're having trouble seeing the entirety of query plans, you can run the following cell to set the limit on displayed rows to 20. **Careful**: Do not set this to `None` and run the actual queries; SQL will return millions of rows and crash your kernel!

In [7]:
# run this cell to remove 10-row limit on display
%config SqlMagic.displaylimit = 20

## Single Table Plans: Impact of limits, orders, selections




<div class="alert alert-success">
It is much easier to see query plans in <b>psql</b>!<br/>
<code>jupysql</code> dataframe visualization removes any whitespace.
</div>

You can also run (after each cell):
```
result = __.DataFrame()
result.style.set_properties(**{'text-align': 'left'})
print(result)
```

In [8]:
def printplans(x):
    result = x.DataFrame()
    result.style.set_properties(**{'text-align': 'left'})
    print(result)

<br/><br/>

Let's start with a simple query. 

In [47]:
%%sql
/* 1a */
EXPLAIN ANALYZE SELECT id FROM actors;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

3 rows affected.

QUERY PLAN
Seq Scan on actors (cost=0.00..13684.88 rows=845888 width=4) (actual time=0.009..66.742 rows=845888 loops=1)
Planning Time: 0.048 ms
Execution Time: 90.349 ms


In [48]:
printplans(_)

                                          QUERY PLAN
0  Seq Scan on actors  (cost=0.00..13684.88 rows=...
1                            Planning Time: 0.048 ms
2                          Execution Time: 90.349 ms


<br/><br/>

Great, now let's see the impact of adding a LIMIT clause. 

In [49]:
%%sql
/* 1b */
EXPLAIN ANALYZE SELECT id FROM actors LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

4 rows affected.

QUERY PLAN
Limit (cost=0.00..0.16 rows=10 width=4) (actual time=0.009..0.010 rows=10 loops=1)
-> Seq Scan on actors (cost=0.00..13684.88 rows=845888 width=4) (actual time=0.008..0.009 rows=10 loops=1)
Planning Time: 0.050 ms
Execution Time: 0.021 ms


In [50]:
printplans(_)

                                          QUERY PLAN
0  Limit  (cost=0.00..0.16 rows=10 width=4) (actu...
1    ->  Seq Scan on actors  (cost=0.00..13684.88...
2                            Planning Time: 0.050 ms
3                           Execution Time: 0.021 ms


Closing postgresql://127.0.0.1:5432/imdb_perf_lecture

<br/><br/>

Wow - that dropped quite a bit!

What if we add an ORDER BY?

In [20]:
%%sql
/* 1c */
EXPLAIN ANALYZE
SELECT id FROM actors
ORDER BY name
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

12 rows affected.

QUERY PLAN
Limit (cost=17366.94..17368.11 rows=10 width=18) (actual time=118.898..121.367 rows=10 loops=1)
-> Gather Merge (cost=17366.94..99611.72 rows=704906 width=18) (actual time=118.896..121.363 rows=10 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Sort (cost=16366.92..17248.05 rows=352453 width=18) (actual time=116.312..116.313 rows=7 loops=3)
Sort Key: name
Sort Method: top-N heapsort Memory: 26kB
Worker 0: Sort Method: top-N heapsort Memory: 25kB
Worker 1: Sort Method: top-N heapsort Memory: 26kB
-> Parallel Seq Scan on actors (cost=0.00..8750.53 rows=352453 width=18) (actual time=0.019..61.395 rows=281963 loops=3)


In [21]:
printplans(_)

                                           QUERY PLAN
0   Limit  (cost=17366.94..17368.11 rows=10 width=...
1     ->  Gather Merge  (cost=17366.94..99611.72 r...
2                                  Workers Planned: 2
3                                 Workers Launched: 2
4           ->  Sort  (cost=16366.92..17248.05 row...
5                                      Sort Key: name
6                 Sort Method: top-N heapsort  Mem...
7                 Worker 0:  Sort Method: top-N he...
8                 Worker 1:  Sort Method: top-N he...
9                 ->  Parallel Seq Scan on actors ...
10                            Planning Time: 0.223 ms
11                         Execution Time: 101.352 ms


<br/> <br/>

The time goes up again - compare this to the original time we had:

In [22]:
%%sql
/* 1a */
EXPLAIN ANALYZE SELECT id FROM actors;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

3 rows affected.

QUERY PLAN
Seq Scan on actors (cost=0.00..13684.88 rows=845888 width=4) (actual time=0.007..67.859 rows=845888 loops=1)
Planning Time: 0.089 ms
Execution Time: 91.475 ms


In [26]:
printplans(_)

                                           QUERY PLAN
0   Limit  (cost=17366.94..17368.11 rows=10 width=...
1     ->  Gather Merge  (cost=17366.94..99611.72 r...
2                                  Workers Planned: 2
3                                 Workers Launched: 2
4           ->  Sort  (cost=16366.92..17248.05 row...
5                                      Sort Key: name
6                 Sort Method: top-N heapsort  Mem...
7                 Worker 0:  Sort Method: top-N he...
8                 Worker 1:  Sort Method: top-N he...
9                 ->  Parallel Seq Scan on actors ...
10                            Planning Time: 0.070 ms
11                         Execution Time: 121.540 ms


<br/> <br/>

Finally, what if we added a selection:

In [24]:
%%sql
/* 1d */
EXPLAIN ANALYZE SELECT id FROM actors
WHERE id > 4000000 AND
name='Tom Hanks';

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

8 rows affected.

QUERY PLAN
Gather (cost=1000.00..11512.90 rows=1 width=4) (actual time=24.200..26.481 rows=0 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Parallel Seq Scan on actors (cost=0.00..10512.80 rows=1 width=4) (actual time=21.615..21.616 rows=0 loops=3)
Filter: ((id > 4000000) AND (name = 'Tom Hanks'::text))
Rows Removed by Filter: 281963
Planning Time: 0.066 ms
Execution Time: 26.499 ms


In [25]:
printplans(_)

                                          QUERY PLAN
0  Gather  (cost=1000.00..11512.90 rows=1 width=4...
1                                 Workers Planned: 2
2                                Workers Launched: 2
3    ->  Parallel Seq Scan on actors  (cost=0.00....
4          Filter: ((id > 4000000) AND (name = 'T...
5                     Rows Removed by Filter: 281963
6                            Planning Time: 0.088 ms
7                          Execution Time: 25.954 ms


Here, the selection right after the sequential scan (see "Filter:" )

<br/> <br/>

## Two-table demo: LIMIT

Let's join two tables, `actors` and `cast_info`. The query planner selects a hash join:

In [27]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info
WHERE actors.id = cast_info.person_id;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

8 rows affected.

QUERY PLAN
Hash Join (cost=29215.48..89168.21 rows=2211936 width=26) (actual time=174.550..1443.178 rows=2211936 loops=1)
Hash Cond: (cast_info.person_id = actors.id)
-> Seq Scan on cast_info (cost=0.00..31907.36 rows=2211936 width=8) (actual time=0.053..139.723 rows=2211936 loops=1)
-> Hash (cost=13684.88..13684.88 rows=845888 width=18) (actual time=174.326..174.328 rows=845888 loops=1)
Buckets: 65536 Batches: 16 Memory Usage: 3114kB
-> Seq Scan on actors (cost=0.00..13684.88 rows=845888 width=18) (actual time=0.005..55.215 rows=845888 loops=1)
Planning Time: 0.179 ms
Execution Time: 1506.325 ms


In [28]:
printplans(_)

                                          QUERY PLAN
0  Hash Join  (cost=29215.48..89168.21 rows=22119...
1       Hash Cond: (cast_info.person_id = actors.id)
2    ->  Seq Scan on cast_info  (cost=0.00..31907...
3    ->  Hash  (cost=13684.88..13684.88 rows=8458...
4          Buckets: 65536  Batches: 16  Memory Us...
5          ->  Seq Scan on actors  (cost=0.00..13...
6                            Planning Time: 0.451 ms
7                        Execution Time: 1454.408 ms


<br/><br/>

Below, we add `LIMIT`. Note the query planner switches to a nested loop join, using an index scan to match `cast_info.person_id` to the indexed attribute `actors.id`! This results in a 10,000x speedup!

Thus, knowing which keywords impact performance is really, really important!

In [30]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info
WHERE actors.id = cast_info.person_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

11 rows affected.

QUERY PLAN
Limit (cost=0.43..4.49 rows=10 width=26) (actual time=0.040..0.078 rows=10 loops=1)
-> Nested Loop (cost=0.43..896026.45 rows=2211936 width=26) (actual time=0.039..0.076 rows=10 loops=1)
-> Seq Scan on cast_info (cost=0.00..31907.36 rows=2211936 width=8) (actual time=0.013..0.014 rows=10 loops=1)
-> Memoize (cost=0.43..0.47 rows=1 width=18) (actual time=0.006..0.006 rows=1 loops=10)
Cache Key: cast_info.person_id
Cache Mode: logical
Hits: 2 Misses: 8 Evictions: 0 Overflows: 0 Memory Usage: 1kB
-> Index Scan using actor_pkey on actors (cost=0.42..0.46 rows=1 width=18) (actual time=0.006..0.006 rows=1 loops=8)
Index Cond: (id = cast_info.person_id)
Planning Time: 0.202 ms


In [31]:
printplans(_)

                                           QUERY PLAN
0   Limit  (cost=0.43..4.49 rows=10 width=26) (act...
1     ->  Nested Loop  (cost=0.43..896026.45 rows=...
2           ->  Seq Scan on cast_info  (cost=0.00....
3           ->  Memoize  (cost=0.43..0.47 rows=1 w...
4                      Cache Key: cast_info.person_id
5                                 Cache Mode: logical
6                 Hits: 2  Misses: 8  Evictions: 0...
7                 ->  Index Scan using actor_pkey ...
8                       Index Cond: (id = cast_inf...
9                             Planning Time: 0.240 ms
10                           Execution Time: 1.545 ms


<br/> <br/>

## Two-table demo: Selection

In [41]:
%%sql
EXPLAIN ANALYZE
SELECT name, movie_id
FROM actors, cast_info
WHERE actors.id = cast_info.person_id AND actors.id > 4000000;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

10 rows affected.

QUERY PLAN
Hash Join (cost=24027.28..81649.00 rows=1171838 width=18) (actual time=162.266..1032.735 rows=634763 loops=1)
Hash Cond: (cast_info.person_id = actors.id)
-> Seq Scan on cast_info (cost=0.00..31907.36 rows=2211936 width=8) (actual time=0.009..210.990 rows=2211936 loops=1)
-> Hash (cost=15799.60..15799.60 rows=448134 width=18) (actual time=161.587..161.588 rows=444781 loops=1)
Buckets: 65536 Batches: 8 Memory Usage: 3335kB
-> Seq Scan on actors (cost=0.00..15799.60 rows=448134 width=18) (actual time=0.234..77.857 rows=444781 loops=1)
Filter: (id > 4000000)
Rows Removed by Filter: 401107
Planning Time: 0.163 ms
Execution Time: 1051.158 ms


In [34]:
printplans(_)

                                          QUERY PLAN
0  Hash Join  (cost=24027.28..81649.00 rows=11718...
1       Hash Cond: (cast_info.person_id = actors.id)
2    ->  Seq Scan on cast_info  (cost=0.00..31907...
3    ->  Hash  (cost=15799.60..15799.60 rows=4481...
4          Buckets: 65536  Batches: 8  Memory Usa...
5          ->  Seq Scan on actors  (cost=0.00..15...
6                             Filter: (id > 4000000)
7                     Rows Removed by Filter: 401107
8                            Planning Time: 0.154 ms
9                        Execution Time: 1121.718 ms


<br/> This, in many cases, will reduce latency, because the selection is reducing data size. Note the predicate is being applied when actors is scanned (aka predicate pushdown)! <br/> <br/>
Try changing 4000000 (4M) to 10000000 (10M) or 100000000 (100M).
<br/><br/>

## [At Home, or in class if time] Three-way joins

Now let's try a three-way join! Feel free to change the LIMIT, but note that removing it entirely can result in a really long query! <br/> <br/>
We've set it to be a large number, 100K, to start.

In [43]:
%%sql 
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info, movies
WHERE actors.id = cast_info.person_id
    AND movies.id = cast_info.movie_id
LIMIT 100000;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

17 rows affected.

QUERY PLAN
Limit (cost=29106.44..41257.64 rows=100000 width=56) (actual time=3015.031..3365.785 rows=100000 loops=1)
-> Gather (cost=29106.44..297883.07 rows=2211936 width=56) (actual time=3015.029..3359.761 rows=100000 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Parallel Hash Join (cost=28106.44..75689.47 rows=921640 width=56) (actual time=3010.941..3088.584 rows=33392 loops=3)
Hash Cond: (cast_info.movie_id = movies.id)
-> Parallel Hash Join (cost=15222.20..45913.91 rows=921640 width=26) (actual time=690.146..1321.540 rows=737312 loops=3)
Hash Cond: (cast_info.person_id = actors.id)
-> Parallel Seq Scan on cast_info (cost=0.00..19004.40 rows=921640 width=8) (actual time=0.011..141.914 rows=737312 loops=3)
-> Parallel Hash (cost=8750.53..8750.53 rows=352453 width=18) (actual time=195.975..195.976 rows=281963 loops=3)


In [ ]:
printplans(_)

<br/><br/>
What if we add an additional selection condition on one of the relations?

Below, note the predicate pushdown in the sequential scan on actors! Again, copy-paste into `psql` if you can't see the whitespace formatting.

In [44]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info, movies
WHERE actors.id = cast_info.person_id
    AND movies.id = cast_info.movie_id
    AND name = 'Tom Hanks';

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

16 rows affected.

QUERY PLAN
Gather (cost=10632.10..32056.15 rows=3 width=56) (actual time=193.571..313.706 rows=65 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Nested Loop (cost=9632.10..31055.85 rows=1 width=56) (actual time=190.948..307.115 rows=22 loops=3)
-> Parallel Hash Join (cost=9631.68..31055.40 rows=1 width=26) (actual time=190.869..306.587 rows=22 loops=3)
Hash Cond: (cast_info.person_id = actors.id)
-> Parallel Seq Scan on cast_info (cost=0.00..19004.40 rows=921640 width=8) (actual time=0.008..131.269 rows=737312 loops=3)
-> Parallel Hash (cost=9631.67..9631.67 rows=1 width=18) (actual time=21.909..21.910 rows=0 loops=3)
Buckets: 1024 Batches: 1 Memory Usage: 40kB
-> Parallel Seq Scan on actors (cost=0.00..9631.67 rows=1 width=18) (actual time=13.857..21.890 rows=0 loops=3)


In [ ]:
printplans(_)

<br/><br/>

Compare with the below predicate pushdown, where the filter is now on movie titles:

In [45]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info, movies
WHERE actors.id = cast_info.person_id
    AND movies.id = cast_info.movie_id
    AND title LIKE 'Snakes on a Plane';

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

16 rows affected.

QUERY PLAN
Gather (cost=9279.46..30704.82 rows=7 width=56) (actual time=205.139..336.325 rows=4 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Nested Loop (cost=8279.46..29704.12 rows=3 width=56) (actual time=192.284..323.602 rows=1 loops=3)
-> Parallel Hash Join (cost=8279.04..29702.75 rows=3 width=38) (actual time=178.881..309.918 rows=1 loops=3)
Hash Cond: (cast_info.movie_id = movies.id)
-> Parallel Seq Scan on cast_info (cost=0.00..19004.40 rows=921640 width=8) (actual time=0.013..110.408 rows=737312 loops=3)
-> Parallel Hash (cost=8279.03..8279.03 rows=1 width=30) (actual time=21.182..21.183 rows=0 loops=3)
Buckets: 1024 Batches: 1 Memory Usage: 40kB
-> Parallel Seq Scan on movies (cost=0.00..8279.03 rows=1 width=30) (actual time=16.196..21.104 rows=0 loops=3)


In [ ]:
printplans(_)

# [At home] Three-way joins with Indexes 

We're now going to see the impact of indexes... Start with a query from before.

In [46]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info, movies
WHERE actors.id = cast_info.person_id
    AND movies.id = cast_info.movie_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

14 rows affected.

QUERY PLAN
Limit (cost=0.86..9.46 rows=10 width=56) (actual time=0.073..0.125 rows=10 loops=1)
-> Nested Loop (cost=0.86..1901460.04 rows=2211936 width=56) (actual time=0.072..0.123 rows=10 loops=1)
-> Nested Loop (cost=0.43..896026.45 rows=2211936 width=26) (actual time=0.027..0.063 rows=10 loops=1)
-> Seq Scan on cast_info (cost=0.00..31907.36 rows=2211936 width=8) (actual time=0.010..0.011 rows=10 loops=1)
-> Memoize (cost=0.43..0.47 rows=1 width=18) (actual time=0.005..0.005 rows=1 loops=10)
Cache Key: cast_info.person_id
Cache Mode: logical
Hits: 2 Misses: 8 Evictions: 0 Overflows: 0 Memory Usage: 1kB
-> Index Scan using actor_pkey on actors (cost=0.42..0.46 rows=1 width=18) (actual time=0.004..0.004 rows=1 loops=8)
Index Cond: (id = cast_info.person_id)


In [ ]:
printplans(_)

<br/><br/>
What if we dropped one of the indexes?

To do so we must drop the primary key constraint on actors.id:

In [43]:
%sql ALTER TABLE actors DROP CONSTRAINT actor_pkey CASCADE;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

++
||
++
++

In [44]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info, movies
WHERE actors.id = cast_info.person_id
    AND movies.id = cast_info.movie_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

15 rows affected.

QUERY PLAN
Limit (cost=16222.62..16228.33 rows=10 width=56) (actual time=625.525..765.128 rows=10 loops=1)
-> Nested Loop (cost=16222.62..1278418.10 rows=2211936 width=56) (actual time=625.524..765.124 rows=10 loops=1)
-> Gather (cost=16222.20..272984.51 rows=2211936 width=26) (actual time=625.481..687.666 rows=10 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Parallel Hash Join (cost=15222.20..50790.91 rows=921640 width=26) (actual time=620.558..620.948 rows=808 loops=3)
Hash Cond: (cast_info.person_id = actors.id)
-> Parallel Seq Scan on cast_info (cost=0.00..19004.40 rows=921640 width=8) (actual time=0.014..186.327 rows=737312 loops=3)
-> Parallel Hash (cost=8750.53..8750.53 rows=352453 width=18) (actual time=129.357..129.358 rows=281963 loops=3)
Buckets: 65536 Batches: 16 Memory Usage: 0kB


In [ ]:
printplans(_)

<br/><br/>
What if we dropped both indexes?

In [45]:
%sql ALTER TABLE movies DROP CONSTRAINT movie_pkey CASCADE;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

++
||
++
++

In [46]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM actors, cast_info, movies
WHERE actors.id = cast_info.person_id
    AND movies.id = cast_info.movie_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

17 rows affected.

QUERY PLAN
Limit (cost=29106.44..29107.70 rows=10 width=56) (actual time=1847.435..1931.973 rows=10 loops=1)
-> Gather (cost=29106.44..307637.07 rows=2211936 width=56) (actual time=1847.434..1931.969 rows=10 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Parallel Hash Join (cost=28106.44..85443.47 rows=921640 width=56) (actual time=1840.619..1840.727 rows=4 loops=3)
Hash Cond: (cast_info.movie_id = movies.id)
-> Parallel Hash Join (cost=15222.20..50790.91 rows=921640 width=26) (actual time=630.536..1252.075 rows=737312 loops=3)
Hash Cond: (cast_info.person_id = actors.id)
-> Parallel Seq Scan on cast_info (cost=0.00..19004.40 rows=921640 width=8) (actual time=0.018..139.780 rows=737312 loops=3)
-> Parallel Hash (cost=8750.53..8750.53 rows=352453 width=18) (actual time=196.594..196.594 rows=281963 loops=3)


In [ ]:
printplans(_)

# Cleanup

We close the connection, then drop the database:

In [ ]:
%sql --close postgresql://127.0.0.1:5432/imdb_perf_lecture

In [ ]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb_perf_lecture'